* Issue of multiple feature on one location 
  * Had to apply set to avoid this issue

In [1]:
import pandas as pd
from collections import Counter
from utils import parse_cdhit, replace_val
import itertools

In [2]:
data = pd.read_table("data/ENA_ML_input_ORFs")

In [3]:
data.head(2)

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein


In [4]:
seq_to_clust = parse_cdhit("data/ENA.40.clstr")

# key is the Feauture_id, value is the cluster to which it belongs.
# what does it look like
list(seq_to_clust.items())[0:4]

[('ENA_KP211958_KP211958.1_65082_87137_88', '0'),
 ('ENA_AY940168_AY940168.2_57144_79082_90', '1'),
 ('ENA_GU071099_GU071099.1_60366_80177_94', '2'),
 ('ENA_GU071103_GU071103.1_54052_75426_80', '2')]

In [5]:
## Add a column to show the cluster to which each feature_id belongs
data["cluster_number"] = replace_val(data.Feature_id, seq_to_clust)
data.cluster_number = data.cluster_number.astype(int)
data.head(2)



,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,cluster_number
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown,57065
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein,43194


In [11]:
# Add a contig sequential number in addition to contig id to make it easier to work with the contig number
contig_id_to_int = {y:x for x,y in enumerate(data.contig_id.tolist())}
#### data.replace({'contig_id': contig_id_to_int})
data["contig_number"] = replace_val(data.contig_id, contig_id_to_int)
data.head(2)

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,cluster_number,contig_number
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown,57065,237296
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein,43194,234989


* Generate a table with all the features per contig


In [31]:
temp = data.groupby("contig_number").apply(lambda x: sorted(set(x["cluster_number"].values)))
subsets = pd.DataFrame({"contig_id": temp.index, "cluster_number": temp.values})
subsets.head(2)

,contig_id,cluster_number
0,82638,"[3929, 6051, 8867, 21058, 25595, 57763]"
1,106079,"[3490, 6292, 18722, 30149]"


* count the occurrences of each feature in all the contigs. 
  * Needed to compute the probability of a pair

In [37]:
feature_counts = pd.DataFrame(Counter(itertools.chain(*subsets.cluster_number.values)).items())
feature_counts.columns = ["cluster_number", "nb_occurrences"]
feature_counts = feature_counts.set_index("cluster_number")
feature_counts.head()

,nb_occurrences
cluster_number,
3929,135
6051,134
8867,135
21058,122
25595,135


In [39]:
# temp
sum([3929 in x for x in subsets.cluster_number.values])

135

* Count the occurrences of each pair of cluster_id that occur on at least 1 contig
* We generate generate all the possible pairs (tuples) of features 


In [42]:
# Generate all the subset of size 2 for each contig
subsets_size_2 = subsets.apply(lambda x: list(itertools.combinations(x["cluster_number"], 2)), axis=1)
subsets_size_2.index = subsets.contig_id
subsets_size_2.head(2)


contig_id
82638     [(3929, 6051), (3929, 8867), (3929, 21058), (3...
106079    [(3490, 6292), (3490, 18722), (3490, 30149), (...
dtype: object

* We make sure the tuple is sorted so it's easy to compute frequencies

In [49]:
co_occurences_2 = []
for item in list(itertools.chain(*subsets_size_2.values)):
    if item[0] > item[1]:
        co_occurences_2.append((item[1], item[0]))
    else:
        co_occurences_2.append((item[0], item[1]))  
co_occurences_2[:5]

[(3929, 6051), (3929, 8867), (3929, 21058), (3929, 25595), (3929, 57763)]

* Count the number of occurrences of each pair of features

In [46]:
co_occurences__2_counts = pd.DataFrame(Counter(co_occurences_2).items())
co_occurences__2_counts.columns = ["combination", "nb_occurrences"]
co_occurences__2_counts = co_occurences__2_counts.sort_values(by="nb_occurrences", ascending=False)
co_occurences__2_counts.head(2)

,combination,nb_occurrences
508135,"(1937, 6879)",178
510515,"(5188, 9330)",175


In [50]:
co_occurences__2_counts.shape

(6987559, 2)

* Only keep those that occur frequently

```python
>>> sum(co_occurences__2_counts.nb_occurrences > 10)
257945
```


In [52]:
co_occurences__2_counts = co_occurences__2_counts[co_occurences__2_counts.nb_occurrences > 10]
co_occurences__2_counts.shape

(257945, 2)

In [57]:
co_occurences__2_counts.iloc[0]

combination       (1937, 6879)
nb_occurrences             178
Name: 508135, dtype: object

In [58]:
subset_size = len(co_occurences__2_counts.iloc[0].combination)

for i in range(subset_size):
    a = co_occurences__2_counts.combination.apply(lambda x: x[i])
    co_occurences__2_counts.loc[:, f"item_{i}"] = feature_counts.loc[a].nb_occurrences.values

co_occurences__2_counts.head(10)

,combination,nb_occurrences,item_0,item_1
508135,"(1937, 6879)",178,264,183
510515,"(5188, 9330)",175,175,175
510497,"(5188, 5262)",175,175,175
510896,"(5262, 9330)",175,175,175
509525,"(3797, 5188)",174,174,175
509150,"(3671, 9330)",174,174,175
510530,"(5188, 14120)",174,175,174
509545,"(3797, 9330)",174,174,175
509130,"(3671, 5188)",174,174,175
509132,"(3671, 5262)",174,174,175


In [59]:
co_occurences__2_counts.tail(10)

,combination,nb_occurrences,item_0,item_1
1422998,"(45536, 61763)",11,12,16
1420845,"(32314, 37658)",11,11,16
1143899,"(5142, 32083)",11,73,11
1143952,"(5482, 22035)",11,22,11
1423103,"(47119, 49743)",11,12,16
1144486,"(8144, 41206)",11,17,11
1422227,"(38128, 48030)",11,11,19
1420909,"(32401, 35715)",11,24,13
1422230,"(38128, 49040)",11,11,16
1143954,"(5482, 23207)",11,22,11


* Compute the Dice Coefficient

In [67]:
co_occurences__2_counts["Dice"] = co_occurences__2_counts["nb_occurrences"] / co_occurences__2_counts.iloc[:,[2,3]].product(axis=1)
co_occurences__2_counts  = co_occurences__2_counts.sort_values(by=["nb_occurrences", "Dice"], ascending=False)
co_occurences__2_counts.head(10)


,combination,nb_occurrences,item_0,item_1,Dice
508135,"(1937, 6879)",178,264,183,0.003684
510515,"(5188, 9330)",175,175,175,0.005714
510497,"(5188, 5262)",175,175,175,0.005714
510896,"(5262, 9330)",175,175,175,0.005714
509525,"(3797, 5188)",174,174,175,0.005714
509150,"(3671, 9330)",174,174,175,0.005714
510530,"(5188, 14120)",174,175,174,0.005714
509545,"(3797, 9330)",174,174,175,0.005714
509130,"(3671, 5188)",174,174,175,0.005714
509132,"(3671, 5262)",174,174,175,0.005714


* What is the top hit?
  (1937, 6879) has `nb_occurrences =178`

In [72]:
data[data.cluster_number == 1937]['annotation_val'].head()

662     Ribonucleoside_diphosphate reductase
1063    Ribonucleoside_diphosphate reductase
1069    Ribonucleoside_diphosphate reductase
1335    Ribonucleoside_diphosphate reductase
1912    Ribonucleoside_diphosphate reductase
Name: annotation_val, dtype: object

In [71]:
data[data.cluster_number == 6879]['annotation_val'].head()

170     Unknown
755     Unknown
2764    Unknown
5850    Unknown
7642    Unknown
Name: annotation_val, dtype: object

* If 6879 is unknown, are any of the other features associated with 1937 konw

In [73]:
co_occurences__2_counts[co_occurences__2_counts.combination.apply(lambda x: 1937 in  x)].head(10)

,combination,nb_occurrences,item_0,item_1,Dice
508135,"(1937, 6879)",178,264,183,0.003684
508145,"(1937, 9330)",172,264,175,0.003723
508127,"(1937, 5262)",172,264,175,0.003723
508125,"(1937, 5188)",172,264,175,0.003723
508140,"(1937, 8643)",171,264,172,0.003766
508162,"(1937, 14438)",171,264,173,0.003744
508118,"(1937, 3671)",171,264,174,0.003723
508120,"(1937, 3797)",171,264,174,0.003723
508160,"(1937, 14120)",171,264,174,0.003723
508133,"(1937, 6277)",170,264,171,0.003766


In [76]:
data[data.cluster_number == 5262]['annotation_val'].head()

1944    DNA primase_helicase
3300    DNA primase_helicase
3366    DNA primase_helicase
4688    DNA primase_helicase
5660    DNA primase_helicase
Name: annotation_val, dtype: object

In [78]:
data[data.cluster_number == 8643]['annotation_val'].head()

226     RecA_like protein
1239              Unknown
1443    RecA_like protein
4673    RecA_like protein
6829    RecA_like protein
Name: annotation_val, dtype: object

In [79]:
data[data.cluster_number == 3797]['annotation_val'].head()

24      Gp17 terminase large subunit
3796    Gp17 terminase large subunit
3927                         Unknown
4671    Gp17 terminase large subunit
4747    Gp17 terminase large subunit
Name: annotation_val, dtype: object

* Repeating the analysis work for larger subsets of features
  * Start with paired what have surprise factor, by either
  1. merging most frequent pairs iterative  add another item   
  2. Start from seed and extending to include all pairs and finding the frequent items


In [ ]:
co_occurences__2_counts.shape

In [ ]:
co_occurences_3 = []

i = 0
for subset in co_occurences__2_counts.combination.values:
    i+=1
    for feature_group in subsets.features.values:
        if subset[0]  in feature_group and  subset[1] in feature_group:
            for feature in feature_group:
                if feature not in subset:
                    temp_group = sorted(list(subset) + [feature])
                    co_occurences_3.append(temp_group)            
    if i%100==0:
        print(i)